<a href="https://colab.research.google.com/github/aynokt/q-macro/blob/main/project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


def calculate_average_growth_rate(data, start_year, end_year, column):
    data = data.loc[(data['year'] >= start_year) & (data['year'] <= end_year)]
    growth_rate = (data[column].iloc[-1] / data[column].iloc[0]) ** (1 / (end_year - start_year)) - 1
    return growth_rate * 100

# Load data
pwt1001 = pd.read_stata('https://dataverse.nl/api/access/datafile/354098')

# Filter and select relevant columns
countries = ['Australia', 'Austria', 'Belgium', 'Canada', 'Denmark', 'Finland', 'France', 'Germany', 'Greece', 'Iceland', 'Ireland', 'Italy', 'Japan', 'Netherlands', 'New Zealand', 'Norway', 'Portugal', 'Spain','Sweden', 'Switzerland', 'United Kingdom', 'United States']
data = pwt1001.loc[pwt1001['country'].isin(countries)][['year', 'country', 'countrycode', 'rgdpna', 'rkna', 'pop', 'emp', 'avh', 'labsh', 'rtfpna', 'hc']]
data = data.loc[(data['year'] >= 1990) & (data['year'] <= 2019)].dropna()

# Calculate additional variables
data['alpha'] = 1 - data['labsh']
data['y_n'] = data['rgdpna'] / data['emp']
data['capital_per_worker'] = data['rkna'] / data['emp']
data['human_capital_per_worker'] = data['hc']
data['hours_per_worker'] = data['avh']

# Calculate average growth rates for each country
results = []
for country in countries:
    country_data = data[data['country'] == country]
    alpha = country_data['alpha'].mean()
    gdp_growth = calculate_average_growth_rate(country_data, 1990, 2019, 'y_n')
    capital_deepening = calculate_average_growth_rate(country_data, 1990, 2019, 'capital_per_worker')
    hc_growth = calculate_average_growth_rate(country_data, 1990, 2019, 'human_capital_per_worker')
    tfp_growth = gdp_growth - alpha * capital_deepening - (1 - alpha) * hc_growth

    results.append({
        'Country': country,
        'Growth Rate': gdp_growth,
        'TFP Growth': tfp_growth,
        'Capital Deepening': capital_deepening,
        'Labor Growth': hc_growth,
        'TFP Share': tfp_growth / gdp_growth if gdp_growth != 0 else np.nan,
        'Capital Share': (alpha * capital_deepening) / gdp_growth if gdp_growth != 0 else np.nan,
        'Labor Share': ((1 - alpha) * hc_growth) / gdp_growth if gdp_growth != 0 else np.nan
    })

# Create a DataFrame from the results
growth_accounting_table = pd.DataFrame(results)

# Calculate the average values for each column
average_values = {
    'Country': 'Average',
    'Growth Rate': growth_accounting_table['Growth Rate'].mean(),
    'TFP Growth': growth_accounting_table['TFP Growth'].mean(),
    'Capital Deepening': growth_accounting_table['Capital Deepening'].mean(),
    'Labor Growth': growth_accounting_table['Labor Growth'].mean(),
    'TFP Share': growth_accounting_table['TFP Share'].mean(),
    'Capital Share': growth_accounting_table['Capital Share'].mean(),
    'Labor Share': growth_accounting_table['Labor Share'].mean(),
}

# Convert the average_values dictionary to a DataFrame
average_values_df = pd.DataFrame([average_values])

# Concatenate the average_values_df with the growth_accounting_table
growth_accounting_table = pd.concat([growth_accounting_table, average_values_df], ignore_index=True)

# Display the updated table
print(growth_accounting_table.to_string(index=False, float_format='{:.2f}'.format))


       Country  Growth Rate  TFP Growth  Capital Deepening  Labor Growth  TFP Share  Capital Share  Labor Share
     Australia         1.28        0.43               1.91          0.14       0.33           0.60         0.06
       Austria         1.04       -0.02               1.94          0.45      -0.02           0.76         0.26
       Belgium         0.95       -0.14               2.10          0.46      -0.14           0.85         0.30
        Canada         0.98        0.04               1.96          0.42       0.04           0.67         0.29
       Denmark         1.36        0.24               2.19          0.51       0.18           0.58         0.24
       Finland         1.45        0.40               1.72          0.58       0.28           0.48         0.24
        France         0.91       -0.04               1.69          0.49      -0.04           0.71         0.33
       Germany         1.04        0.28               1.68          0.24       0.27           0.58      